# modeling1-Longformer-datapreprocessing

In [1]:
from datasets import load_dataset
import json
import os
import argparse
import transformers
import torch
import numpy as np
import random
from torch import nn
from torch.utils.data import Dataset
from transformers import (
    AutoModelForSequenceClassification, 
    AutoModel, 
    AutoConfig, 
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    TrainingArguments,
    set_seed,
)
from util.arguments import ModelArguments, DataTrainingArguments 
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

import json
from copy import deepcopy
from pprint import pprint
# from transformers.models.longformer impor LongformerForSequenceClassification, LongformerSequenceClassifierOutput
# from transformers.modeling_outputs import LongformerSequenceClassifierOutput
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [22]:
model_name = 'roberta-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [23]:
tokenizer

PreTrainedTokenizerFast(name_or_path='roberta-large', vocab_size=50265, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})

In [28]:
def preprocess_function(examples):
        # Tokenize the texts
    return tokenizer(
        examples["premise"],
        examples["hypothesis"],
        max_length=200,
        truncation=True,
    )

In [19]:
train_dataset = load_dataset(
                "xnli",
                'de',
                split="validation",
                use_auth_token=None,
            )

Found cached dataset xnli (/home/philhoon/.cache/huggingface/datasets/xnli/de/1.1.0/818164464f9c9fd15776ca8a00423b074344c3e929d00a2c1a84aa5a50c928bd)


In [25]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [29]:
train_dataset = train_dataset.map(
                preprocess_function,
                batched=True,
                desc="Running tokenizer on validation dataset",
            )

Running tokenizer on validation dataset:   0%|          | 0/3 [00:00<?, ?ba/s]

In [33]:
type(train_dataset[0]['label'])

int

In [11]:
train_dataset[0]

{'premise': 'und er hat gesagt, Mama ich bin daheim.',
 'hypothesis': 'Er rief seine Mutter an, sobald er aus dem Schulbus stieg.',
 'label': 1}

In [27]:
# parser = HfArgumentParser(
#         (ModelArguments, DataTrainingArguments, TrainingArguments)
#     )
# # parser.parse_args_into_dataclasses([])
# # args = ["--foo", "1", "--baz", "quux", "--bar", "0.5"]
# args = ["--model_name_or_path", 'allenai/longformer-large-4096', '--output_dir', './']
# # (example,) = parser.parse_args_into_dataclasses(args, look_for_args_file=False)
# # args = parser.parse_args(args=[])
# model_args, data_args, training_args = parser.parse_args_into_dataclasses(args)
# # print(model_args.model_name_or_path)

In [28]:
# model_args
# data_args
# training_args

In [29]:
parser = argparse.ArgumentParser()
MY_DICT = {'key' : 'value'}

In [30]:
parser.add_argument('-s', '--seed', type=int, default=42, help='random seed (default : 42)')
parser.add_argument('-m', '--model', type=str, default='BaseModel', help='model type (default:BaseModel)')
parser.add_argument('--train-dataset', type=str, default='NQ', help='train dataset (default:Natural Question)')
parser.add_argument('--eval_dataset', type=str, default='NQ', help='eval dataset (default:Natural Question)')
parser.add_argument('--output_dir', type=str, default='output directory', help='output directory')
# parser.add_argument('--cache_directory', type = str, default = '', help='default transformer cahce directory')

_StoreAction(option_strings=['--output_dir'], dest='output_dir', nargs=None, const=None, default='output directory', type=<class 'str'>, choices=None, help='output directory', metavar=None)

In [31]:
args = parser.parse_args([])

In [32]:
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

In [33]:
# print(args)

In [34]:
seed_everything(args.seed)

In [35]:
# config file are updated when executing .save_pretrained('./')
# model architecture depends on model_name
model = AutoModelForSequenceClassification.from_pretrained('allenai/longformer-base-4096', num_labels=2)

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias'

In [16]:
# TEST1
# model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-large-4096')
# model = AutoModel.from_pretrained('allenai/longformer-large-4096')
# type(model)

# TEST2
# Custom LongformerForSequenceClassification
# class CustomSequenceClassification(nn.Module):
#     def __init__(self, model_name, num_labels):
#         super(CustomSequenceClassification, self).__init__()
#         self.num_labels = num_labels
#         self.model_name = model_name
        
#         # Base Model Configuration
#         self.model = AutoModel.from_pretrained(self.model_name, add_pooling_layer = False)
#         self.config = self.model.config
        
#         # Custom Layers for Sequence Classification
#         self.dense = nn.Linear(self.config.hidden_size, self.config.hidden_size)
#         self.dropout = nn.Dropout(self.config.hidden_dropout_prob)
#         self.out_proj = nn.Linear(self.config.hidden_size, self.num_labels)
        
#     def forward(self, 
#                 input_ids=None, 
#                 attention_mask=None, 
#                 labels=None):
        
#         outputs = self.model(
#             input_ids=input_ids, 
#             attention_mask=attention_mask,
#         )
        
#         # outputs[0] -> last hidden layer 
#         sequence_output = outputs[0]
        
#         # [batch, sequence, hidden] -> [:, 0, :] -> hidden state of <s> or [CLS] token 
#         hidden_states = hidden_states[:, 0, :]
#         hidden_states = self.dropout(hidden_states)
#         hidden_states = self.dense(hidden_states)
#         hidden_states = torch.tanh(hidden_states)
#         hidden_states = self.dropout(hidden_states)
#         logits = self.out_proj(hidden_states)
        
#         loss = None
#         if labels is not None:
#             # single_label_classification
#             loss_fct = CrossEntropyLoss()
#             loss = loss_fct(logits.view(-1, self.num_labels), lablels.view(-1))
            
#         return SequenceClassifierOutput(
#             loss=loss, 
#             logits=logits, 
#             hidden_states=outputs.hidden_states,
#             attentions=outputs.attentions,
#             global_attentions=outputs.gloabl_attentions
#         )

# TEST3
# class CustomSequenceClassification(AutoModelForSequenceClassification):
#     def __init__(self, model_name, num_labels):
#         super().__init__()
#         self.model_name = model_name
#         self.num_labels = num_labels
#         self.model = AutoModelForSequenceClassification.from_pretrained(self.model_name, num_labels=self.num_labels)
        
#     def forward(self, 
#                 input_ids=None, 
#                 attention_mask=None, 
#                 labels=None):
        
#         outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        
#         return outputs

## nq

In [17]:
def open_json(file):
    with open(file , 'r') as f: 
        data = json.load(f)
    return data

In [21]:
nq_test_file = '/data/philhoon-relevance/KILT/kilt-dpr-retrieval/nq-dev-multikilt.json'
nq_dpr_dev = open_json(nq_test_file)

In [22]:
nq_train_file = '/data/philhoon-relevance/KILT/kilt-dpr-retrieval/nq-train-multikilt.json'
nq_dpr_train = open_json(nq_train_file)

In [23]:
print(len(nq_dpr_dev))
print(len(nq_dpr_train))

2837
76945


In [ ]:
# pprint(nq_dpr_dev[0])

In [24]:
model_name = 'allenai/longformer-large-4096'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# pprint(nq_dpr_train[-1])

In [ ]:
# # cut_off = 0
# # negative_sample_size = 5
# # position = 0
# total_questions = len(nq_dpr_train)
# cnt = 0 
# for samples in nq_dpr_train:
# #     negatives = []
# #     pprint(samples)
# #     question = samples['question']
# #     answer = answer['question']
# #     negative_samples = []  

# #     positive_sample = []
# #     if len(samples['positive_ctxs']) < 1:
# # #         pprint(samples)
# # #         break
# #         cnt += 1
# #     if len(samples['hard_negative_ctxs']) < 5:
# # #         pprint(samples)
# # #         break
# #         cnt += 1
#     if len(samples['hard_negative_ctxs']) < 5 or len(samples['positive_ctxs']) < 1:
#         cnt += 1
# print(cnt)

In [ ]:
# import random

# mylist = ["apple", "banana", "cherry"]
# random.shuffle(mylist)

# print(mylist)
# new_lst = deepcopy(mylist)
# new_lst[1] = 'pos'
# print(new_lst)
# print(mylist)

In [ ]:
# cut_off = 0
# instances = []
# sample_size = 4
# position = 2

# total_quetions = len(nq_dpr_train)
# for samples in nq_dpr_train:
# #     negatives = []
# #     pprint(samples)
# #     question = samples['question']
#     answer = samples['answers'] 
#     question = samples['question']
#     negative_samples = []
#     # sample_size = 4
#     # position 2
#     # random.shuffle(negative_samples)
#     # negative_samples = [n3, n1, n2, n4]
#     # adding positive context at position 2
#     # positive_samples = [n3, n1, p1, n3] 
    
#     # 'hard_negative_ctxs' should be at least equal to sample_size
#     if len(samples['hard_negative_ctxs']) < sample_size or len(samples['positive_ctxs']) < 1:
#         cut_off += 1
#     else:
#         cnt_negative_sample = 0
#         for negative_sample in samples['hard_negative_ctxs']:
#             if cnt_negative_sample > sample_size - 1:
#                 break
#             ng_s = negative_sample['text'].replace('\n', ' ')
#             negative_samples.append(ng_s)
#             cnt_negative_sample += 1
            
#         random.shuffle(negative_samples)
        
#         # replace 1 negative_sample with positive_sample
#         positive_sample = samples['positive_ctxs'][0]['text'].replace('\n', ' ')
#         positive_samples = deepcopy(negative_samples)
#         positive_samples[position] = positive_sample
        
#         negative_template={
#             'text' : negative_samples,
#             'labels' : 0,
#             'answer' : answer,
#             'question' : question,
#         }
#         positive_template={
#             'text' : positive_samples,
#             'labels' : 1,
#             'answer' : answer,
#             'question' : question,
#             'pos' : position,
#         }
#         instances.append(negative_template)
#         instances.append(positive_template)

# print(cut_off)

    
# #         pprint(samples['hard_negative_ctxs'])
# #         pprint(samples['psg_id'])
# #     pprint(samples['positive_ctx'])
# # print(cut_off)

In [ ]:
# cnt = 0
# sample_size = 5

# for instance in nq_dpr_train:
# #     print(len(instance['hard_negative_ctxs']) >= 5)
#     if len(instance['hard_negative_ctxs']) < sample_size:
#         cnt += 1
# print(cnt)

In [25]:
def preprocessing_data(json_file, sample_size:int, position:int):
    """
    sample_size : one to five
        e.g.)
            positive_sample = 1 positive passage + n-1 negative passage
            negative_sample = n negative passage
    cut_off : number of questions discarded when there is not enough negative passages
    position : position of positive passage (1 ~ n)
        e.g.) n = 2, position = 1
            instance = [negative passage, positive passage]
    """
    cut_off = 0
    instances = []
    sample_size = sample_size
    position = position
    total_questions = len(json_file) 
    
    for idx, samples in enumerate(json_file):
        answer = samples['answers'] 
        question = samples['question']
        negative_samples = []
    
        # 'hard_negative_ctxs' should be at least equal to sample_size
        # 'positive_ctx' which contains the answer should be at least one
        if len(samples['hard_negative_ctxs']) < sample_size or len(samples['positive_ctxs']) < 1:
            cut_off += 1
        else:
            cnt_negative_sample = 0
            for negative_sample in samples['hard_negative_ctxs']:
                if cnt_negative_sample > sample_size - 1:
                    break
                ng_s = negative_sample['text'].replace('\n', ' ')
                negative_samples.append(ng_s)
                cnt_negative_sample += 1
            
            # 'hard_negative_ctxs' sorted by its score, so shuffle them
            random.shuffle(negative_samples)
            
            # replace 1 negative_sample with one positive_sample in designated position
            positive_sample = samples['positive_ctxs'][0]['text'].replace('\n', ' ')
            positive_samples = deepcopy(negative_samples)
            positive_samples[position-1] = positive_sample 
            
            negative_template={
            'text' : negative_samples,
            'labels' : 0,
            'answer' : answer,
            'question' : question,
            }
            positive_template={
                'text' : positive_samples,
                'labels' : 1,
                'answer' : answer,
                'question' : question,
                'pos' : position,
            }
            instances.append(negative_template)
            instances.append(positive_template)
    
    return instances, cut_off, total_questions

In [26]:
# pprint(nq_dpr_dev[15])

In [37]:
sample_size = 5
position = 1
instances, cut_off, total_questions = preprocessing_data(nq_dpr_train, sample_size, position)

In [38]:
print(total_questions)
print(cut_off)

76945
2801


In [46]:
print(len(instances))
dev_instance = instances[:3000]
train_instance = instances[3000:]

148288


In [47]:
print(len(dev_instance))
print(len(train_instance))

3000
145288


In [31]:
instances[0]

{'text': ['4:38 2. "Lover Man (Oh Where Can You Be?)" (Davis, Ramirez, Sherman) – 5:25 3. "Ah, That\'s Freedom" – 10:08 4. "Don\'t Ever Leave Me" – 4:28 5. "Willow Weep for Me" (Ronell) – 6:15 6. "Mean What You Say" – 5:51 7. "Once Around" – 12:45 8. "Polka Dots and Moonbeams" (Van Heusen, Burke) – 4:02',
  ', 3(2):2200, 2008. - Misha Gromov. "Topological invariants of dynamical systems and spaces of holomorphic maps". I. Math. Phys. Anal. Geom., 2(4):323–415, 1999. - Elon Lindenstrauss and Benjamin Weiss. "Mean topological dimension". Israel J. Math., 115:1–24, 2000. External links. What is Mean Dimension?',
  '1. "Introduction (Mel Lewis & Alan Grant)" – 1:50 2. "Big Dipper" – 5:10 3. "Polka Dots and Moonbeams" (Van Heusen, Burke) – 3:47 4. "Once Around" – 12:37 5. "All My Yesterdays" – 4:08 6. "Morning Reverend" – 4:50 7. "Low Down" – 4:25 8. "Lover Man" (Davis, Ramirez, Sherman) – 5:08 9. "Mean What',
  'Track listings appear in a slightly different order on the reissued version as

In [ ]:
# def checking_instance(instances, sample_size, position):
#     for instance in instances:
#         if len(instance['text']) != sample_size:
#             print('not right sample_size')
            
#         label = instance['labels']
#         answer_lst = instance['answer']
        
#         if not label:
#             for answer in answer_lst:
#                 for text in instance['text']:
#                        if answer in text:
#                             print('-------')
#                             print('answer in negative context')
#                             print(f'answer : {answer}')
#                             print(f'instance ')
#                             pprint(instance)
#                             print('-------')
#         else:
#             for answer in answer_lst:
#                 for idx, text in enumerate(instance['text'], 1):
#                     if answer in text and idx != position:
#                         print('-------')
#                         print(f'answer in wrong position context')
#                         print(f'answer : {answer}')
#                         print(f'idx : {idx}')
#                         print(f'position : {position}')
#                         print(f'instance ')
#                         pprint(instance)
#                         print('-------')
# checking_instance(instances, sample_size, position)

In [ ]:
# instances_text = deepcopy(instances)

In [ ]:
# class CustomDataset_2(torch.utils.data.Dataset):
#     def __init__(self, instances, cls_token, sep_token):
#         self.instances = instances
#         self.cls_token = cls_token
#         self.sep_token = sep_token
    
#     def __len__(self):
#         return len(self.instances)
    
#     def __getitem__(self, idx):
#         input_ = [f'{self.cls_token} ' + self.instances[idx]['question']] + self.instances[idx]['text']
#         input_txt =  f' {self.sep_token} '.join(input_) + f' {self.sep_token}'
#         label = self.instances[idx]['labels']
        
#         return {'text' : input_txt, 'label' : label}
# dataset2 = CustomDataset_2(instances, tokenizer.cls_token, tokenizer.sep_token)

In [ ]:
# checking tokenizer length

# from tqdm.notebook import tqdm
# check_token_size = []
# for i in tqdm(range(0, len(dataset2))):
#     token_lenght = len(tokenizer(dataset2[i]['text'])['input_ids'])
# #     token_length = dataset[i]['text'].size()[1]
#     check_token_size.append(token_lenght)
# print(f'percentage under 1024 {sum((i <= 1024 for i in check_token_size))/len(instances)}')
# import pandas as pd
# df = pd.DataFrame (check_token_size, columns = ['token_length'])
# df.describe()

In [48]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, instances, tokenizer, max_length):
        self.instances = instances
        self.tokenizer = tokenizer
        self.sep_token = tokenizer.sep_token
        self.max_length = max_length
    
    def __len__(self):
        return len(self.instances)
    
    def __getitem__(self, idx):
        input_ = [' ' + self.instances[idx]['question']] + self.instances[idx]['text']
        input_txt =  f' { self.sep_token } '.join(input_) + ' '
        
        output = self.tokenizer(
            input_txt, 
            # return_tensors="pt", will be applied later through collator
            # padding=True, will be padded later through collate
            truncation=True, 
            add_special_tokens=True, 
            max_length=self.max_length)
        
        item = {key : val for key, val in output.items()}
        item['labels'] = torch.tensor(self.instances[idx]['labels'])
        
        return item

In [50]:
dataset = CustomDataset(dev_instance, tokenizer, 1024)

In [54]:
# d

In [55]:
# dataset[0]['input_ids']

In [ ]:
# tokenizer.decode(dataset[3]['input_ids'])

In [ ]:
# tokenizer.convert_ids_to_tokens(dataset[3]['input_ids'])

In [ ]:
# from tqdm.notebook import tqdm
# check_token_size = []
# for i in tqdm(range(0, len(dataset))):
#     token_length = dataset[i]['input_ids'].size()[1]
#     check_token_size.append(token_length)
# print(f'percentage under 1024 {sum((i <= 1024 for i in check_token_size))/len(instances)}')
# import pandas as pd
# df = pd.DataFrame (check_token_size, columns = ['token_length'])
# df.describe()

In [ ]:
# dataset[1]['input_ids'][:10]

In [51]:
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

In [52]:
data_collator = DataCollatorWithPadding(
        tokenizer, 
        pad_to_multiple_of=8,
    )

In [ ]:
# Trainer implement dataloader inside
# data_loader = DataLoader(
#     dataset, 
#     batch_size=8, 
#     shuffle=True, 
#     sampler=None,
#     batch_sampler=None, 
#     num_workers=1, 
#     collate_fn=data_collator,
#     pin_memory=False, 
#     drop_last=False, 
#     timeout=0,
#     worker_init_fn=None)

In [ ]:
# k = next(iter(data_loader))

In [ ]:
# k

In [ ]:
# k['input_ids'].size()

In [ ]:
# for i in k['input_ids'].numpy():
#     print(tokenizer.convert_ids_to_tokens(i))
#     print('=====')

In [ ]:
# for i in data_loader:
#     print(i['input_ids'].size())

In [ ]:
# k['input_ids'].size()

In [ ]:
# DataLoader(dataset, batch_size=1, shuffle=False, sampler=None,
#            batch_sampler=None, num_workers=0, collate_fn=None,
#            pin_memory=False, drop_last=False, timeout=0,
#            worker_init_fn=None)

In [ ]:
# from datasets import Dataset

# def gen():
#     for idx in len(dataset):
#         yield dataset[idx]  # this has to be a dictionary
#     ## or if it's an IterableDataset
#     # for ex in torch_dataset:
#     #     yield ex

# dset = Dataset.from_generator(gen)

# on-the-fly transformation
# class Encoder():
#     def __init__(tokenizer):
#         self.tokenizer = tokenizer
        
#     def encode(examples):
#         return self.tokenizer(
#             examples['text'],
#             return_tensors="pt",
#             padding=True,
#             truncation=True,
#             add_special_tokens=False,
#             max_length = max_length
#         )

# def encode(examples):
#     return tokenizer(
#         examples['text'],
#         return_tensors="pt",
#         padding=True,
#         truncation=True,
#         add_special_tokens=False,
#         max_length = max_length
#     )

In [ ]:
from transformers import TrainingArguments